# This notebook demonstrates how to use kepler_kinematics to calculate stellar velocities

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from astropy.io import fits
import pymc3 as pm

import kepler_kinematics as kek

Let's load the Gaia-Kepler crossmatched catalog.

In [3]:
with fits.open("../../data/kepler_dr2_1arcsec.fits") as data:
    gaia = pd.DataFrame(data[1].data, dtype="float64")

In [4]:
gaia_rv = gaia.iloc[gaia.radial_velocity.values != 0.]

Select a star.

In [38]:
star_index = 2
df = gaia_rv.iloc[star_index]

First, let's calculate the 3D velocities of the star using its radial velocity.

In [39]:
# for i, row in enumerate(df):
xyz, vxyz = kek.simple_calc_vxyz(df["ra"], df["dec"],
                                 1./df["parallax"],
                                 df["pmra"], df["pmdec"],
                                 df["radial_velocity"])

vx, vy, vz = vxyz
x, y, z = xyz

Now let's infer the 3D velocities of the star. First we'll create arrays of position, parallax, and proper motion.

In [40]:
pos = [float(df["ra"]), float(df["dec"]), float(df["parallax"])]
pos_err = [float(df["ra_error"]), float(df["dec_error"]),
           float(df["parallax_error"])]
proper = [float(df["pmra"]), float(df["pmdec"])]
proper_err = [float(df["pmra_error"]), float(df["pmdec_error"])]

Get the prior that's built-in for Kepler stars.

In [41]:
mu, cov = kek.get_prior()

/Users/rangus/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3254: DtypeWarning: Columns (1,2,3,20,21,22,28,30,54,55,56,74,76,155,156,176) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Run the velocity inference code.

In [42]:
trace = kek.run_pymc3_model(pos, pos_err, proper, proper_err, mu, cov)

optimizing logp for variables: [vxyzD]


message: Optimization terminated successfully.
logp: -2823.3997376390475 -> -5.676444099942984


Multiprocess sampling (4 chains in 4 jobs)
NUTS: [vxyzD]


Sampling 4 chains for 1_500 tune and 1_000 draw iterations (6_000 + 4_000 draws total) took 23 seconds.


Pull out the results from the samples.

In [44]:
import arviz as az

az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
vxyzD[0],16.987,8.761,0.710,33.344,0.129,0.097,4642.0,4081.0,4651.0,2796.0,1.0
vxyzD[1],224.922,23.034,179.929,265.880,0.338,0.240,4631.0,4604.0,4638.0,2909.0,1.0
vxyzD[2],2.058,4.231,-6.073,9.684,0.062,0.062,4633.0,2350.0,4641.0,2864.0,1.0
vxyzD[3],-0.167,0.019,-0.204,-0.134,0.000,0.000,4281.0,4277.0,4297.0,2775.0,1.0
vxyz[0],16.987,8.761,0.710,33.344,0.129,0.097,4642.0,4081.0,4651.0,2796.0,1.0
vxyz[1],224.922,23.034,179.929,265.880,0.338,0.240,4631.0,4604.0,4638.0,2909.0,1.0
vxyz[2],2.058,4.231,-6.073,9.684,0.062,0.062,4633.0,2350.0,4641.0,2864.0,1.0
log_D,-0.167,0.019,-0.204,-0.134,0.000,0.000,4281.0,4277.0,4297.0,2775.0,1.0
D,0.846,0.016,0.815,0.875,0.000,0.000,4275.0,4267.0,4297.0,2775.0,1.0
xyz[0],-7.826,0.006,-7.836,-7.816,0.000,0.000,4275.0,4275.0,4297.0,2775.0,1.0


In [31]:
# trace.posterior.to_dataframe()
print(np.median(trace.posterior["D"].values))
print(np.median(trace.posterior["vxyz"].values, axis=(0, 1)))

0.6716677948207352
[ 52.73752921 220.48851724   0.6870397 ]


In [10]:
flat_samples = pm.trace_to_dataframe(trace)

params_inferred = np.median(flat_samples, axis=0)
upper = np.percentile(flat_samples, 84, axis=0)
lower = np.percentile(flat_samples, 16, axis=0)
errp = upper - params_inferred
errm = params_inferred - lower
std = np.std(flat_samples, axis=0)

results = pd.DataFrame(dict({"kepid": df["kepid"],
                             "vx": params_inferred[0], "vx_errp": errp[0], "vx_errm": errm[0], "vx_err": std[0],
                             "vy": params_inferred[1], "vy_errp": errp[1], "vy_errm": errm[1], "vy_err": std[1],
                             "vz": params_inferred[2], "vz_errp": errp[2], "vz_errm": errm[2], "vz_err": std[2],
                             "lndistance": params_inferred[3], "lndistance_errp": errp[3],
                             "lndistance_errm": errm[3], "lndistance_err": std[3]
                            }), index=[0])

In [11]:
print(f"{vx:.1f}, {float(results.vx.values):.1f} +/- {float(results.vx_err.values):.1f} km / s")
print(f"{vy:.1f}, {float(results.vy.values):.1f} +/- {float(results.vy_err.values):.1f} km / s")
print(f"{vz:.1f}, {float(results.vz.values):.1f} +/- {float(results.vz_err.values):.1f} km / s")

50.4 km / s, 52.8 +/- 8.7 km / s
215.2 km / s, 220.7 +/- 22.7 km / s
-0.3 km / s, 0.7 +/- 4.2 km / s


In [12]:
print(f"{vx:.1f}, {float(results.vx.values):.1f} +/- {float(results.vx_err.values):.1f} km / s")
print(f"{vy:.1f}, {float(results.vy.values):.1f} +/- {float(results.vy_err.values):.1f} km / s")
print(f"{vz:.1f}, {float(results.vz.values):.1f} +/- {float(results.vz_err.values):.1f} km / s")

50.4 km / s, 52.8 +/- 8.7 km / s
215.2 km / s, 220.7 +/- 22.7 km / s
-0.3 km / s, 0.7 +/- 4.2 km / s
